In [1]:
import gym
import minihack
import matplotlib.pyplot as plt
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from collections import deque
import time
from datetime import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical


/home/evan/anaconda3/envs/nle/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# TODO
1. Figure out why its not learning (could be because we are using the whole glyphs, and not just glyphs_crop)
1. Figure out how to save and load policy object
1. Figure out how to run on a Cluster or on Colab
1. Improvements to model/algorithm (maybe just include baseline? Or change to actor-critic)
1. Hyperparameter Tuning
1. Set up to run with multiple seeds for statistical significance purposes

# Useful URLs

https://minihack.readthedocs.io/en/latest/index.html

https://minihack.readthedocs.io/en/latest/getting-started/observation_spaces.html#options

https://discuss.pytorch.org/t/pytorch-multiple-input-and-output/140020/2

https://goodboychan.github.io/python/reinforcement_learning/pytorch/udacity/2021/05/12/REINFORCE-CartPole.html

# Utility Functions


In [2]:
def get_shapes(observation):
    '''
    Args: 
        observation: (dict) An environment observation
    Ret:
        shapes: (list) A list of the shapes of the numpy arrays in the observation dict
    '''
    shapes = []
    for key in observation.keys():
        if not key == "pixel":
            thing = observation[key]
            shapes.append(thing.shape)
    return shapes

def save_gif(gif,path=None):
    '''
    Args:
        gif: a list of image objects
        path: the path to save the gif to, defaults to <current datetime>.gif
    '''
    if path is None:
        path = '/home/evan/RL_Assignment/repo/video'+datetime.now().strftime("%d-%m-%Y_%H:%M:%S") + '.gif'
    gif[0].save(path, save_all=True,optimize=False, append_images=gif[1:], loop=0)
    print("Saved Video")


# Init Env

In [3]:
save_dir = "/home/evan/RL_Assignment/ttyrec_files"
device = torch.device("cpu")
# obs_keys = ('glyphs', 'chars','screen_descriptions', 'glyphs_crop', 'chars_crop', 'screen_descriptions_crop','blstats', 'message', 'inv_strs', 'pixel')
obs_keys = ('glyphs_crop','inv_glyphs', 'pixel')

env = gym.make(
    "MiniHack-Quest-Hard-v0",
    savedir=save_dir,
    observation_keys=obs_keys
)
    

# Policy

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [42]:
class Policy(nn.Module):
    def __init__(self, env, hidden_size=100):
        super(Policy, self).__init__()

        # initialising state
        state = env.reset()
        action_size = env.action_space.n

        # Getting the shapes of the state space
        shapes = get_shapes(state)
        self.obs_keys = obs_keys

        input_size = 136 # self.get_input_shape(shapes)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, action_size)
    
    def get_input_shape(self,shapes):
        length = 0
        for shape in shapes:
            length +=sum(shape)
        return length

    def forward(self, state):
        '''
        Args: state, an observation from env.step(). 
            state is a dictionary, with keys "glyphs" and "inv_glyphs"
        Ret: action probabilites

        '''
        
        glyph_features = torch.from_numpy(state['glyphs_crop']).float().to(device)
        glyph_features = torch.flatten(glyph_features)
        inv_features = torch.from_numpy(state['inv_glyphs']).to(device)
        x = torch.cat((glyph_features.float(),inv_features.float()))
        
        x = torch.tanh(self.fc1(x))
        action_probabilities = torch.tanh(self.fc2(x)).float()
        # print(action_probabilities)
        # we just consider 1 dimensional probability of action
        return F.softmax(action_probabilities,dim=0)
    
    def act(self, state):
        '''
        Args: state, an observation from env.step(). 
            state is a dictionary, with keys "glyphs" and "inv_glyphs"
        Ret: action,probability(action|policy,state)

        '''
        probs = self.forward(state).cpu()
        # print(probs)
        model = Categorical(probs)
        action = model.sample()
        return action.item(), model.log_prob(action)

In [6]:
# class Policy(nn.Module):
#     def __init__(self, env, hidden_size=100):
#         super(Policy, self).__init__()

#         # initialising state
#         state = env.reset()
#         self.glyphState = np.array([
#             state['glyphs'],
#             state['glyphs'],
#             state['glyphs']
#         ])
#         action_size = env.action_space.n

#         # Getting the shapes of the state space
#         shapes = get_shapes(state)
#         self.obs_keys = obs_keys

#         self.glyphConv2D1 = nn.Conv2d(3,1,(3,5),stride=(2,3))
#         self.glyphMaxPool1 = nn.MaxPool2d(2, stride=2)

#         self.fc1 = nn.Linear(115, hidden_size)
#         self.fc2 = nn.Linear(hidden_size, action_size)
        
#     def forward(self, state):
#         '''
#         Args: state, an observation from env.step(). 
#             state is a dictionary, with keys "glyphs" and "inv_glyphs"
#         Ret: action probabilites

#         '''
#         self.glyphState = np.insert(self.glyphState[:-1,:,:],0,state['glyphs'],axis=0)
        
#         glyph_features = torch.from_numpy(self.glyphState ).float().unsqueeze(0).to(device)
#         glyph_features = F.relu(self.glyphConv2D1(glyph_features))
#         glyph_features = torch.flatten(self.glyphMaxPool1(glyph_features))
#         inv_features = torch.from_numpy(state['inv_glyphs']).to(device)
#         x = torch.cat((glyph_features.float(),inv_features.float()))
        
#         x = F.relu(self.fc1(x))
#         action_probabilites = F.relu(self.fc2(x))
#         # we just consider 1 dimensional probability of action
#         return F.softmax(action_probabilites, dim=-1)
    
#     def act(self, state):
#         '''
#         Args: state, an observation from env.step(). 
#             state is a dictionary, with keys "glyphs" and "inv_glyphs"
#         Ret: action,probability(action|policy,state)

#         '''
#         probs = self.forward(state).cpu()
#         model = Categorical(probs)
#         action = model.sample()
#         return action.item(), model.log_prob(action)

# REINFORCE

In [43]:
def reinforce(policy, optimizer, n_episodes=1000, max_t=1000, gamma=1.0, print_every=100):
    '''
    Args:
        policy: a pytorch model, takes in a state and outputs an action
        optimizer: a pytorch optimizer
        n_episodes: (int) number of episodes to train for
        max_t: (int) max time steps per episode
        gamma: (float, [0,1]) discount factor
        print_every: (int) number of episodes between print of update
    
    Ret:
        scores: list of total rewards per episode
        gif: a list of images corresponding to the best scoring episode
    '''
    print("Starting Reinforce")
    best_frames = None
    latest_frames=None
    best_score = None
    scores_deque = deque(maxlen=100)
    scores = []
    for e in range(1, n_episodes):
        frames = []
        saved_log_probs = []
        rewards = []
        state = env.reset()
        # Collect trajectory
        for t in range(max_t):
            # Sample the action from current policy
            action, log_prob = policy.act(state)
            # action = np.random.randint(0, high=85)
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action) 
            frame = state["pixel"]
            frames.append(frame)
            rewards.append(reward)
            if done:
                break
        # Calculate total expected reward
        scores_deque.append(sum(rewards))
        score = sum(rewards)
        scores.append(score)

        # Keep track of the gif of the best scoring episode
        latest_frames = frames
        if best_score is None or score > best_score: 
            best_score=score
            best_frames = frames
        
        # Recalculate the total reward applying discounted factor
        discounts = [gamma ** i for i in range(len(rewards) + 1)]
        R = sum([a * b for a,b in zip(discounts, rewards)])
        
        # Calculate the loss 
        policy_loss = []
        for log_prob in saved_log_probs:
            # Note that we are using Gradient Ascent, not Descent. So we need to calculate it with negative rewards.
            policy_loss.append(-log_prob * R)
        # After that, we concatenate whole policy loss in 0th dimension
        # print(policy_loss[0])
        policy_loss = torch.stack(policy_loss).sum()
        
        # Backpropagation
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        
        if e % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(e, np.mean(scores_deque)))
        if np.mean(scores_deque) >= 195.0:
            print('Environment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(e - 100, np.mean(scores_deque)))
            break

    gif = []
    for image in best_frames:
        gif.append(Image.fromarray(image, "RGB"))
    last_gif =[]
    for image in latest_frames:
        last_gif.append(Image.fromarray(image, "RGB"))
    print("Done!")

    return scores,gif,last_gif

# Run

In [44]:
policy = Policy(env).to(device)
optimizer = optim.Adam(policy.parameters(), lr=1e-2)

In [36]:
x = torch.tensor([ 4.8951e+02,  1.8116e+03,  3.6915e+02, -2.8129e+01,  2.7978e+03]).float()#,
        # -5.9301e+01,  5.2306e+02,  4.5036e+02,  8.7317e+02,  1.1032e+03,
        # -2.1825e+02, -8.3054e+01, -1.0596e+02,  9.2776e+02,  7.9150e+02,
        #  1.4746e+03, -3.9986e+00, -1.1751e+02, -4.8166e+01,  8.0139e+02,
        #  5.5373e+02, -2.2087e+02, -1.6877e+01, -2.5839e+02,  8.8373e+02,
        # -6.3782e+00,  4.9773e+02, -7.6848e+00, -3.6406e+02,  1.0373e+03,
        # -3.5552e+01, -1.8686e+02, -2.4091e+01,  3.3224e+02, -1.0081e+02,
        # -1.2646e+02,  1.8410e+03, -2.8760e+02, -5.7098e+01,  9.4553e+02,
        # -1.7202e+01,  8.1378e+02, -8.1557e+01, -4.2418e+02, -2.4669e+01,
        #  2.8906e+02, -3.5567e+02, -3.2394e+02, -2.1453e+02,  1.5906e+03,
        # -1.3053e+02,  8.4127e+02, -1.8754e+02,  5.1593e+02,  7.5083e+02,
        # -4.6466e+02,  9.2046e-01,  6.9515e+02,  5.9445e+02,  6.8192e+02,
        # -2.3958e+02,  6.1424e+02,  6.8675e+02, -3.9655e+01, -3.1689e+01,
        # -4.3349e+01,  5.1774e+02,  6.2190e+02,  1.1146e+02, -1.7059e+02,
        #  1.8702e+02, -3.5262e+01, -1.2733e+02,  1.4611e+03, -9.3537e+01,
        # -1.2122e+02,  1.2088e+03, -2.3834e+02,  3.3737e+02,  3.6786e+02,
        #  1.1546e+03,  3.2439e+02,  8.0422e+02,  9.0449e+01,  1.5224e+03]).float()

x = torch.randn(4)
print(x)

tensor([ 0.1013,  0.9376,  0.1056, -0.5770])


In [37]:
float(3.6915e+02),float(2.7978e+03),float(1.5224e+03)

(369.15, 2797.8, 1522.4)

In [41]:
F.softmax(torch.tensor([ float(3.6915e+02),float(2.7978e+03),float(1.5224e+03)]),dim=0)

tensor([0., 1., 0.])

In [30]:
state = env.reset()
# while True:
print(policy.act(state))

tensor([ 4.8951e+02,  1.8116e+03,  3.6915e+02, -2.8129e+01,  2.7978e+03,
        -5.9301e+01,  5.2306e+02,  4.5036e+02,  8.7317e+02,  1.1032e+03,
        -2.1825e+02, -8.3054e+01, -1.0596e+02,  9.2776e+02,  7.9150e+02,
         1.4746e+03, -3.9986e+00, -1.1751e+02, -4.8166e+01,  8.0139e+02,
         5.5373e+02, -2.2087e+02, -1.6877e+01, -2.5839e+02,  8.8373e+02,
        -6.3782e+00,  4.9773e+02, -7.6848e+00, -3.6406e+02,  1.0373e+03,
        -3.5552e+01, -1.8686e+02, -2.4091e+01,  3.3224e+02, -1.0081e+02,
        -1.2646e+02,  1.8410e+03, -2.8760e+02, -5.7098e+01,  9.4553e+02,
        -1.7202e+01,  8.1378e+02, -8.1557e+01, -4.2418e+02, -2.4669e+01,
         2.8906e+02, -3.5567e+02, -3.2394e+02, -2.1453e+02,  1.5906e+03,
        -1.3053e+02,  8.4127e+02, -1.8754e+02,  5.1593e+02,  7.5083e+02,
        -4.6466e+02,  9.2046e-01,  6.9515e+02,  5.9445e+02,  6.8192e+02,
        -2.3958e+02,  6.1424e+02,  6.8675e+02, -3.9655e+01, -3.1689e+01,
        -4.3349e+01,  5.1774e+02,  6.2190e+02,  1.1

In [45]:
scores,gif,last_gif = reinforce(policy, optimizer, n_episodes=5,max_t=1000,print_every=1)

Starting Reinforce


/home/evan/anaconda3/envs/nle/lib/python3.8/site-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Episode 1	Average Score: -9.31
Episode 2	Average Score: -9.41
Episode 3	Average Score: -9.45
Episode 4	Average Score: -8.67
Done!


In [46]:
save_gif(gif)
save_gif(last_gif)

Saved Video
Saved Video


In [44]:
import pathlib
pathlib.Path().resolve()

PosixPath('/home/evan/RL_Assignment/repo')